# Titanic Data Set - Deep Learnin Classification
### Author: Eren ERİŞ

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.api._v2.keras import activations
from sklearn.metrics import mean_squared_error , r2_score

import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df = pd.concat([df_train,df_test])

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [6]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
df["Title"]=df["Name"].str.extract(" ([A-Za-z]+)\.",expand=True)
df["Title"].value_counts()

Title
Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Major         2
Ms            2
Lady          1
Sir           1
Mme           1
Don           1
Capt          1
Countess      1
Jonkheer      1
Dona          1
Name: count, dtype: int64

In [8]:
df["Title"]=df["Title"].replace(["Ms","Mle"],"Miss")
df["Title"]=df["Title"].replace(["Mme","Countess","Lady","Dona"],"Mrs")
df["Title"]=df["Title"].replace(["Rev","Jonkheer","Don","Sir","Major","Dr","Capt","Col"],"Mr")

In [9]:

df[['Fare']] = df[['Fare']].fillna(value=df[['Fare']].mean())
df[['Embarked']] = df[['Embarked']].fillna(value=df['Embarked'].value_counts().idxmax())

In [10]:
# Boş değerleri doldur
df['Age'].fillna(df['Title'].map(df.groupby('Title')['Age'].mean()), inplace=True)

In [11]:
df['Family_Size'] = df['Parch'] + df['SibSp'] + 1

In [12]:
df = df.drop(['PassengerId','Name', 'Ticket', 'Cabin'], axis=1)

In [13]:
df=pd.get_dummies(df,drop_first=True)

In [14]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Family_Size,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mlle,Title_Mr,Title_Mrs
0,0.0,3,22.0,1,0,7.2500,2,True,False,True,False,False,True,False
1,1.0,1,38.0,1,0,71.2833,2,False,False,False,False,False,False,True
2,1.0,3,26.0,0,0,7.9250,1,False,False,True,True,False,False,False
3,1.0,1,35.0,1,0,53.1000,2,False,False,True,False,False,False,True
4,0.0,3,35.0,0,0,8.0500,1,True,False,True,False,False,True,False


In [15]:
train = df[:891]
test=df[891:]

In [16]:
x=train.drop('Survived',axis=1)
y=train['Survived']

In [17]:
# x ve y dizilerindeki verileri ondalık sayılara dönüştür
x = x.astype('float32')
y = y.astype('float32')

In [18]:
x.head()

,Pclass,Age,SibSp,Parch,Fare,Family_Size,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mlle,Title_Mr,Title_Mrs
0,3.0,22.0,1.0,0.0,7.250000,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,38.0,1.0,0.0,71.283302,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3.0,26.0,0.0,0.0,7.925000,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,1.0,35.0,1.0,0.0,53.099998,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,3.0,35.0,0.0,0.0,8.050000,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [19]:
y.head()

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Survived, dtype: float32

In [20]:
model=Sequential() # model tanımlama yapıldı
model.add(Dense(8,activation='relu')) # girdi fonksiyonu tanımlandı
model.add(Dense(120,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(6,activation='relu'))
model.add(Dense(2,activation='relu'))
model.add(Dense(1,activation='sigmoid'))#Çıkış fonksiyonu tıkladı. Classification olduğu için activation sigmoid olabilir


In [31]:
# Compiling the NN
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

# Train the NN
model.fit(x, y, batch_size = 32, epochs = 1500)

Epoch 1/1500
28/28 [==============================] - 1s 1ms/step - loss: 0.2517 - accuracy: 0.8866
Epoch 2/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2335 - accuracy: 0.8990
Epoch 3/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2414 - accuracy: 0.8900
Epoch 4/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.2506 - accuracy: 0.8956
Epoch 5/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.2308 - accuracy: 0.8956
Epoch 6/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.2323 - accuracy: 0.9035
Epoch 7/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.2408 - accuracy: 0.9046
Epoch 8/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2452 - accuracy: 0.8967
Epoch 9/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.2400 - accuracy: 0.9012
Epoch 10/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2479 - accuracy: 0.8967

28/28 [==============================] - 0s 1ms/step - loss: 0.2424 - accuracy: 0.8945
Epoch 83/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2269 - accuracy: 0.9046
Epoch 84/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2323 - accuracy: 0.8979
Epoch 85/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2405 - accuracy: 0.8889
Epoch 86/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2296 - accuracy: 0.9024
Epoch 87/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2320 - accuracy: 0.8990
Epoch 88/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2228 - accuracy: 0.9102
Epoch 89/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2225 - accuracy: 0.9113
Epoch 90/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2195 - accuracy: 0.9080
Epoch 91/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2168 - accuracy: 0.9035
Epoc

28/28 [==============================] - 0s 1ms/step - loss: 0.2213 - accuracy: 0.9001
Epoch 163/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2183 - accuracy: 0.9046
Epoch 164/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2129 - accuracy: 0.9024
Epoch 165/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2160 - accuracy: 0.9158
Epoch 166/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2097 - accuracy: 0.9113
Epoch 167/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2023 - accuracy: 0.9158
Epoch 168/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2222 - accuracy: 0.9158
Epoch 169/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2277 - accuracy: 0.9080
Epoch 170/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2104 - accuracy: 0.9125
Epoch 171/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2145 - accuracy: 0.

28/28 [==============================] - 0s 1ms/step - loss: 0.2075 - accuracy: 0.9102
Epoch 243/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2070 - accuracy: 0.9147
Epoch 244/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2051 - accuracy: 0.9125
Epoch 245/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2020 - accuracy: 0.9102
Epoch 246/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2008 - accuracy: 0.9259
Epoch 247/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2179 - accuracy: 0.9080
Epoch 248/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2247 - accuracy: 0.8979
Epoch 249/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2198 - accuracy: 0.9080
Epoch 250/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2160 - accuracy: 0.9068
Epoch 251/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.2130 - accuracy: 0.

28/28 [==============================] - 0s 1ms/step - loss: 0.1904 - accuracy: 0.9203
Epoch 323/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1823 - accuracy: 0.9226
Epoch 324/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1912 - accuracy: 0.9192
Epoch 325/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1821 - accuracy: 0.9226
Epoch 326/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2162 - accuracy: 0.9024
Epoch 327/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2128 - accuracy: 0.9080
Epoch 328/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2034 - accuracy: 0.9158
Epoch 329/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2140 - accuracy: 0.9091
Epoch 330/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2150 - accuracy: 0.9068
Epoch 331/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1961 - accuracy: 0.

...

28/28 [==============================] - 0s 1ms/step - loss: 0.1675 - accuracy: 0.9293
Epoch 1122/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1706 - accuracy: 0.9270
Epoch 1123/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1557 - accuracy: 0.9383
Epoch 1124/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1692 - accuracy: 0.9259
Epoch 1125/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1655 - accuracy: 0.9192
Epoch 1126/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1504 - accuracy: 0.9349
Epoch 1127/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1586 - accuracy: 0.9304
Epoch 1128/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1479 - accuracy: 0.9394
Epoch 1129/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1420 - accuracy: 0.9405
Epoch 1130/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1527 - acc

28/28 [==============================] - 0s 1ms/step - loss: 0.1589 - accuracy: 0.9304
Epoch 1201/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1476 - accuracy: 0.9371
Epoch 1202/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1622 - accuracy: 0.9360
Epoch 1203/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1425 - accuracy: 0.9416
Epoch 1204/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1416 - accuracy: 0.9327
Epoch 1205/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1592 - accuracy: 0.9360
Epoch 1206/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1574 - accuracy: 0.9304
Epoch 1207/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1435 - accuracy: 0.9371
Epoch 1208/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1365 - accuracy: 0.9428
Epoch 1209/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1370 - acc

28/28 [==============================] - 0s 2ms/step - loss: 0.1667 - accuracy: 0.9282
Epoch 1280/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1558 - accuracy: 0.9349
Epoch 1281/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1464 - accuracy: 0.9450
Epoch 1282/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1473 - accuracy: 0.9360
Epoch 1283/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1442 - accuracy: 0.9349
Epoch 1284/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1605 - accuracy: 0.9428
Epoch 1285/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1682 - accuracy: 0.9293
Epoch 1286/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1500 - accuracy: 0.9383
Epoch 1287/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1439 - accuracy: 0.9394
Epoch 1288/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1397 - acc

28/28 [==============================] - 0s 2ms/step - loss: 0.1360 - accuracy: 0.9405
Epoch 1359/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1361 - accuracy: 0.9349
Epoch 1360/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1390 - accuracy: 0.9338
Epoch 1361/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1452 - accuracy: 0.9360
Epoch 1362/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1610 - accuracy: 0.9270
Epoch 1363/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1520 - accuracy: 0.9394
Epoch 1364/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1644 - accuracy: 0.9304
Epoch 1365/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1699 - accuracy: 0.9304
Epoch 1366/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.2703 - accuracy: 0.9113
Epoch 1367/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1809 - acc

28/28 [==============================] - 0s 1ms/step - loss: 0.1353 - accuracy: 0.9383
Epoch 1438/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1305 - accuracy: 0.9439
Epoch 1439/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1528 - accuracy: 0.9360
Epoch 1440/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.2555 - accuracy: 0.9259
Epoch 1441/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1800 - accuracy: 0.9248
Epoch 1442/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1673 - accuracy: 0.9248
Epoch 1443/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1550 - accuracy: 0.9304
Epoch 1444/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1551 - accuracy: 0.9349
Epoch 1445/1500
28/28 [==============================] - 0s 1ms/step - loss: 0.1602 - accuracy: 0.9293
Epoch 1446/1500
28/28 [==============================] - 0s 2ms/step - loss: 0.1612 - acc

In [32]:
score=model.evaluate(x,y)

28/28 [==============================] - 0s 1ms/step - loss: 0.1350 - accuracy: 0.9439


In [33]:
x_test=test.drop('Survived',axis=1)

In [34]:
x_test = x_test.astype('float32')

In [35]:
test['Survived'] = model.predict(x_test)

14/14 [==============================] - 0s 1ms/step


C:\Users\EREN\AppData\Local\Temp\ipykernel_30476\113575540.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Survived'] = model.predict(x_test)


In [36]:
test['Survived']

0      0.041029
1      0.000010
2      0.000002
3      0.440923
4      0.003241
         ...   
413    0.078049
414    1.000000
415    0.207229
416    0.078049
417    0.999890
Name: Survived, Length: 418, dtype: float32

In [37]:
test['Survived'] = (test['Survived'] > 0.5).astype(int)

C:\Users\EREN\AppData\Local\Temp\ipykernel_30476\946910664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Survived'] = (test['Survived'] > 0.5).astype(int)


In [38]:
sonuc = df_test[["PassengerId"]]
sonuc["Survived"] = test['Survived'].astype("int64")

C:\Users\EREN\AppData\Local\Temp\ipykernel_30476\3234773822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sonuc["Survived"] = test['Survived'].astype("int64")


In [39]:
test['Survived']

0      0
1      0
2      0
3      0
4      0
      ..
413    0
414    1
415    0
416    0
417    1
Name: Survived, Length: 418, dtype: int32

In [40]:
sonuc.to_csv("titanicPredict.csv",index=False)